# Merge Bugianen and serve as pmtiles

It's all in the title :)
Pre-requisite: 
* Bugianen files downloaded in the working directory below (links in https://tartamillo.wordpress.com/bugianen/)
* s3 object store setup done in aws-cli (for the very end)

In [1]:
%cd ~/Downloads/dwnmaps/mapdata/mbtiles/Bugianen

/home/me/Downloads/dwnmaps/mapdata/mbtiles/Bugianen


In [3]:
import sys
sys.path.append('../../eslope/development/src')
from mbt_util import mbt_merge, mbt_info

In [97]:
mbt_merge('Bugianen 2005 Cervino.mbtiles',
          'Bugianen 2005 Cuneese.mbtiles',
          'Bugianen 2005 Germanasca.mbtiles',
          'Bugianen 2005 Gran Paradiso.mbtiles',
          'Bugianen 2005 Lanzo.mbtiles',
          'Bugianen 2005 Monregalese.mbtiles',
          'Bugianen 2005 Monte Bianco.mbtiles',
          'Bugianen 2005 Monte Rosa.mbtiles',
          'Bugianen 2005 Monviso.mbtiles',
          'Bugianen 2005 Ossola.mbtiles',
          'Bugianen 2005 Susa.mbtiles',
          'Bugianen 2005 Verbano.mbtiles',
          dest='Bugianen.mbtiles',
          description='https://tartamillo.wordpress.com/bugianen/')

cp Bugianen 2005 Cervino.mbtiles Bugianen.mbtiles
<<>> Bugianen 2005 Cervino : zoom = 12 16 ; n = 8525 ; bounds = 7.295,45.706,7.734,46.012 ; format = jpg
<< Bugianen 2005 Cuneese : zoom = 12 16 ; n = 22847 ; bounds = 6.768,44.088,7.646,44.590 ; format = jpg
>> Bugianen : zoom = 12 16 ; n = 31372 ; bounds = 6.768,44.088,7.734,46.012 ; format = jpg
<< Bugianen 2005 Germanasca : zoom = 12 16 ; n = 15004 ; bounds = 6.592,44.778,7.471,45.089 ; format = jpg
>> Bugianen : zoom = 12 16 ; n = 46376 ; bounds = 6.592,44.088,7.734,46.012 ; format = jpg
<< Bugianen 2005 Gran Paradiso : zoom = 12 16 ; n = 19411 ; bounds = 6.943,45.398,7.910,45.768 ; format = jpg
>> Bugianen : zoom = 12 16 ; n = 64082 ; bounds = 6.592,44.088,7.910,46.012 ; format = jpg
<< Bugianen 2005 Lanzo : zoom = 12 16 ; n = 9207 ; bounds = 7.031,45.213,7.646,45.460 ; format = jpg
>> Bugianen : zoom = 12 16 ; n = 70902 ; bounds = 6.592,44.088,7.910,46.012 ; format = jpg
<< Bugianen 2005 Monregalese : zoom = 12 16 ; n = 14322 ; b

In [2]:
# Don't keep duplicate indexes
import sqlite3
db = sqlite3.connect('Bugianen.mbtiles')
try:
    db.cursor().execute('DROP INDEX IF EXISTS tiles_id')
finally:
    db.commit()
    db.close()


In [70]:
mbt_info('Bugianen.mbtiles')

'zoom = 12 16 ; n = 46376 ; bounds = 6.768,44.088,7.646,44.590 ; format = jpg'

In [81]:
!command -v pmtiles-convert &> /dev/null || pip install pmtiles

In [98]:
!pmtiles-convert Bugianen.{mb,pm}tiles

Num tiles: 152742
Num unique tiles: 151947
Num leaves: 3


In [82]:
# configure your s3
# pip install awscli-plugin-endpoint
# aws configure set plugins.endpoint awscli_plugin_endpoint
# aws configure

In [100]:
!aws s3 cp --acl public-read ./Bugianen.mbtiles s3://maki/

upload: ./Bugianen.mbtiles to s3://maki/Bugianen.mbtiles          


In [1]:
!aws s3 cp --acl public-read ./Bugianen.pmtiles s3://maki/


The user-provided path ./Bugianen.pmtiles does not exist.


In [3]:
!aws s3api put-bucket-cors --bucket maki --cors-configuration \
    '{"CORSRules": [{"AllowedOrigins": ["*"], "AllowedHeaders": ["*"], "AllowedMethods": ["GET", "HEAD"], "MaxAgeSeconds": 3000, "ExposeHeaders": ["Etag"]}]}'